In [1]:
import os
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

In [2]:
import numpy as np
import glob
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import ramanspy as rp

In [2]:
import sys  
sys.path.insert(1, '../../../')

import autoencoders
import unmix

### Parameters

In [3]:
num_endmembers = 20
epochs = 20
seed = 42

results_folder = 'results'

results = {}

### Load the data

In [4]:
volume = rp.SpectralVolume.from_image_stack([rp.load.witec(matlab_file) for matlab_file in sorted(glob.glob(f'raw data/*.mat'))])

### Preprocess the data

In [5]:
preprocessing_pipeline = rp.preprocessing.Pipeline([
    rp.preprocessing.misc.Cropper(region=(700, 1800)),
    rp.preprocessing.despike.WhitakerHayes(),
    rp.preprocessing.denoise.SavGol(window_length=7, polyorder=3),
    rp.preprocessing.baseline.ASLS(),
    rp.preprocessing.normalise.MinMax(a=0, b=1, pixelwise=False),
])

volume_preprocessed = preprocessing_pipeline.apply(volume)

### Define models

In [20]:
def init_models(n_end):
    return [
        ('vca', rp.analysis.unmix.VCA(n_endmembers=n_end, abundance_method='nnls')),
        ('dense_ae_linear', autoencoders.DenseAE(input_dim=len(volume_preprocessed.spectral_axis), bottleneck_dim=n_end, decoder_type='linear', asc=False, encoder_hidden_dims=[128])),
        ('deep_dense_ae_linear', autoencoders.DenseAE(input_dim=len(volume_preprocessed.spectral_axis), bottleneck_dim=n_end, asc=False, decoder_type='linear', encoder_hidden_dims=[512, 256, 128, 64, 32])),
    ]

### Perform analysis

In [ ]:
methods = init_models(20)

for name, method in methods:
    print(f'Unmixing with {name}')

    unmix.set_seeds(int(seed))

    endmembers, abundance_maps = unmix.unmixing(method, volume_preprocessed, epochs=epochs)

    results[name] = (endmembers, abundance_maps)